[Leetcode discussion](https://leetcode.com/discuss/interview-question/4589621/Hard-question..)

Given an array A of size N can have negative or positive elements
In one operation you can swap two adjacent elements but swaps causes negate their signs
Find the minimum number of operations required to make the array non-decreasing order from left to right, return -1 if not posible
Constraints-
N <= 2e5
−10^9 ≤ A[i] ≤ 10^9

Example - 1
N=6
A={-2, 7 ,-1, -8, 2, 8}

output
3

Explanation-
It is possible to perform 3 swaps as follows:
1. Swap the 2nd and 3rd elements so that the array becomes [−2, 1, −7, −8, 2, 8].
2. Swap the 4th and 5th elements so that the array becomes [−2, 1, −7, −2, 8, 8].
3. Swap the 3rd and 4th elements so that the array becomes [−2, 1, 2, 7, 8, 8].

Example-2
N=4
A={1, -1, 1, -1}
Output
-1

In [ ]:
import collections
import contextlib
import itertools
import functools
import sys
import random
import time

DEBUG = True


@contextlib.contextmanager
def time_it(name: str = "", file=sys.stderr):
    begin = time.perf_counter()
    yield
    end = time.perf_counter()
    name = name + ":\t" if name else ""
    delta_ms = 1000 * (end - begin)
    if DEBUG:
        print(f"{name}{delta_ms:.3f} ms", file=file)


def leave_only_least_significant_bit(x):
    return x & ~(x - 1)


class BinaryIndexedTree:
    """Aka Fenwick tree."""

    def __init__(self, size):
        self.size = size
        self.tree = [0] * (size + 1)

    def update(self, index, value):
        index += 1
        while index <= self.size:
            self.tree[index] += value
            index += leave_only_least_significant_bit(index)

    def get_prefix_sum(self, index):
        index += 1
        prefix_sum = 0
        while index:
            prefix_sum += self.tree[index]
            index -= leave_only_least_significant_bit(index)
        return prefix_sum

    def get_range_sum(self, begin, end):
        return self.get_prefix_sum(end - 1) - self.get_prefix_sum(begin - 1)

In [ ]:
def get_equivalence_class(array, pos) -> bool:
    """All elements are divided into two equivalence classes by this bool value."""
    return pos & 1 == (array[pos] < 0)


class Solution:
    @time_it("Total")
    def find_min_ops(self, array):
        self.preprocess(array)

        min_ops_from_false = min(self.min_ops_for_abs_value(0, False))
        min_ops_from_true = min(self.min_ops_for_abs_value(0, True))
        for i in range(1, len(self.sorted_abs)):
            ops_false_to_false, ops_false_to_true = self.min_ops_for_abs_value(i, False)
            ops_true_to_false, ops_true_to_true = self.min_ops_for_abs_value(i, True)
            min_ops_from_false, min_ops_from_true = (
                min(
                    ops_false_to_false + min_ops_from_false,
                    ops_false_to_true + min_ops_from_true,
                ),
                min(
                    ops_true_to_false + min_ops_from_false,
                    ops_true_to_true + min_ops_from_true,
                ),
            )

        return -1 if min_ops_from_false == float("inf") else min_ops_from_false

    @time_it("Preprocessing")
    def preprocess(self, array):
        self.sorted_abs = sorted(set(abs(i) for i in array))
        sorted_abs_values = collections.defaultdict(list)
        for i, value in enumerate(array):
            sorted_abs_values[abs(value)].append(i)

        self.bit = BinaryIndexedTree(len(array))
        self.abs_classes = {
            i: [collections.deque(), collections.deque()] for i in self.sorted_abs
        }
        for abs_value in self.sorted_abs:
            for pos in sorted_abs_values[abs_value]:
                abs_class = get_equivalence_class(array, pos)
                pos_among_less_or_equal = self.bit.get_prefix_sum(pos)
                self.bit.update(pos, 1)
                self.abs_classes[abs_value][abs_class].append(pos_among_less_or_equal)

        self.abs_classes_cumulative_sizes = list(
            itertools.accumulate(len(sorted_abs_values[i]) for i in self.sorted_abs)
        )  # Sizes of unprocessed subarrays for each abs value.
        self.bit = BinaryIndexedTree(len(array))  # Reset bit.

    def min_ops_for_abs_value(self, sorted_abs_index, begin_class):
        # Zero abs value is an edge case.
        if not self.sorted_abs[sorted_abs_index]:
            return 0, 0

        abs_values = SameAbsValues(
            abs_classes=self.abs_classes[self.sorted_abs[sorted_abs_index]],
            size=self.abs_classes_cumulative_sizes[sorted_abs_index],
            begin_class=begin_class,
            bit=self.bit,
        )
        ops = 0

        while abs_values.can_move_to_begin():
            ops += abs_values.move_to_begin()

        while abs_values.can_move_to_end():
            ops += abs_values.move_to_end()

        min_ops = [float("inf"), float("inf")]
        if abs_values.has_unplaced_abs_values():
            abs_values.reset()
            return min_ops

        min_ops[abs_values.begin_class] = ops

        if abs_values.begin_class != abs_values.end_class:
            # Move one at a time: ...010  101... -> ...01  0101...
            while abs_values.can_remove_from_begin():
                ops -= abs_values.remove_from_begin()
                ops += abs_values.move_to_end()
                min_ops[abs_values.begin_class] = min(
                    min_ops[abs_values.begin_class], ops
                )

        else:
            # Move two at a time: ...010  010... -> ...0  01010...
            while abs_values.can_remove_from_begin():
                ops -= abs_values.remove_from_begin()
                if not abs_values.can_remove_from_begin():
                    break
                ops -= abs_values.remove_from_begin()
                ops += abs_values.move_to_end()
                ops += abs_values.move_to_end()
                min_ops[abs_values.begin_class] = min(
                    min_ops[abs_values.begin_class], ops
                )

        abs_values.reset()
        return min_ops


class SameAbsValues:
    def __init__(self, abs_classes, size, begin_class, bit):
        self.abs_classes = abs_classes
        self.size = size
        self.begin_class = begin_class
        self.end_class = not begin_class if size & 1 else begin_class
        self.bit = bit
        self.begin = []
        self.end = []

    def can_move_to_begin(self):
        return bool(self.abs_classes[self.begin_class])

    def can_move_to_end(self):
        return bool(self.abs_classes[self.end_class])

    def has_unplaced_abs_values(self):
        return self.abs_classes[False] or self.abs_classes[True]

    def can_remove_from_begin(self):
        return bool(self.begin)

    def can_remove_from_end(self):
        return bool(self.end)

    def move_to_begin(self):
        pos = self.abs_classes[self.begin_class].popleft()
        ops = pos - self.bit.get_prefix_sum(pos)
        self.bit.update(pos, 1)
        self.begin_class = not self.begin_class
        self.begin.append(pos)
        return ops

    def move_to_end(self):
        pos = self.abs_classes[self.end_class].pop()
        ops = self.size - 1 - self.bit.get_range_sum(pos, self.size) - pos
        self.bit.update(pos, 1)
        self.end_class = not self.end_class
        self.end.append(pos)
        return ops

    def remove_from_begin(self):
        pos = self.begin.pop()
        self.bit.update(pos, -1)
        ops = pos - self.bit.get_prefix_sum(pos)
        self.begin_class = not self.begin_class
        self.abs_classes[self.begin_class].appendleft(pos)
        return ops

    def remove_from_end(self):
        pos = self.end.pop()
        self.bit.update(pos, -1)
        ops = self.size - 1 - self.bit.get_range_sum(pos, self.size) - pos
        self.end_class = not self.end_class
        self.abs_classes[self.end_class].append(pos)
        return ops

    def reset(self):
        """Need to reset the binary indexed tree and the abs classes after processing."""
        while self.can_remove_from_begin():
            self.remove_from_begin()

        while self.can_remove_from_end():
            self.remove_from_end()

In [8]:
class SolutionNSquared:
    def find_min_ops(self, array):
        min_ops = self.find_min_ops_recursive(tuple(array))
        return -1 if min_ops == float("inf") else min_ops

    @functools.cache
    def find_min_ops_recursive(self, array):
        max_abs_value = max((abs(i) for i in array), default=0)
        if not max_abs_value:
            return 0

        n = len(array)
        left_ops = float("inf")
        for i in range(n):
            if abs(array[i]) == max_abs_value and i & 1 == (array[i] > 0):
                suffix = [-j for j in array[:i]] + list(array[i + 1 :])
                left_ops = i + self.find_min_ops_recursive(tuple(suffix))
                break

        right_ops = float("inf")
        for i in reversed(range(n)):
            if abs(array[i]) == max_abs_value and (n - 1 - i) & 1 == (array[i] < 0):
                prefix = list(array[:i]) + [-j for j in array[i + 1 :]]
                right_ops = n - 1 - i + self.find_min_ops_recursive(tuple(prefix))
                break

        return min(left_ops, right_ops)


def swap(array, i):
    array[i], array[i + 1] = -array[i + 1], -array[i]


def generate_valid_test_case(n=200_000, max_abs_value=10**9):
    array = sorted(random.randint(-max_abs_value, max_abs_value) for _ in range(n))
    for _ in range(min(1_000_000, n**2)):
        swap(array, random.randint(0, n - 2))

    return array


def test():
    global DEBUG
    DEBUG = False

    assert Solution().find_min_ops([0, 0]) == 0
    assert Solution().find_min_ops([1, -1]) == -1
    assert Solution().find_min_ops([1, 3, 2]) == 1
    assert Solution().find_min_ops([3, 2, 0]) == 2
    assert Solution().find_min_ops([2, -1, -2]) == 2
    assert Solution().find_min_ops([-2, 7, -1, -8, 2, 8]) == 3

    for _ in range(100):
        array = generate_valid_test_case(n=10, max_abs_value=3)
        expected = SolutionNSquared().find_min_ops(array)
        received = Solution().find_min_ops(array)
        if expected != received:
            raise ValueError(
                f"Wrong answer\nArray:\t{array}\nExpected:\t{expected}\nReceived:\t{received}"
            )

    print("Ok")


def stress_test():
    global DEBUG
    DEBUG = True

    print("Most abs values are different:")
    Solution().find_min_ops(generate_valid_test_case(n=200_000, max_abs_value=10**9))

    print("Most abs values are same:")
    Solution().find_min_ops(generate_valid_test_case(n=200_000, max_abs_value=1))


test()
stress_test()

Ok
Most abs values are different:


Preprocessing:	957.235 ms
Total:	3269.784 ms


Most abs values are same:


Preprocessing:	365.363 ms
Total:	1969.567 ms
